<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-necessary-libraries/packages" data-toc-modified-id="Import-necessary-libraries/packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import necessary libraries/packages</a></span><ul class="toc-item"><li><span><a href="#Libraries/packages" data-toc-modified-id="Libraries/packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Libraries/packages</a></span></li><li><span><a href="#User-defined-functions" data-toc-modified-id="User-defined-functions-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>User-defined functions</a></span></li></ul></li><li><span><a href="#Import-data" data-toc-modified-id="Import-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import data</a></span></li><li><span><a href="#Data-understanding---EDA" data-toc-modified-id="Data-understanding---EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data understanding - EDA</a></span><ul class="toc-item"><li><span><a href="#Descriptive-statistics" data-toc-modified-id="Descriptive-statistics-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Descriptive statistics</a></span></li><li><span><a href="#Data-visualizations" data-toc-modified-id="Data-visualizations-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Data visualizations</a></span></li></ul></li></ul></div>

## Import necessary libraries/packages

### Libraries/packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import aictools.core as aic
metrics = ['name', 'type', 'hist', 'min', 'max', 'mean', 'median', 'missing', 'unique']
import seaborn as sns
sns.set()

%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Set max number of columns to show when listing dataframe (e.g. by .head())
maxcols_tovis = 50
if pd.get_option("display.max_columns")<maxcols_tovis:
    pd.set_option("display.max_columns", maxcols_tovis)

### User-defined functions

In [4]:
def plot_barcharts_for_columns(df,cols,maxvals=15, rowheight=5):
    nrows, ncols = int(np.ceil(len(cols)/2)), 2
    nplots = len(cols)
    fig, ax = plt.subplots(figsize=(20,rowheight*nrows))
    i = 1
    for col in cols:  
        temp_data = df[col].dropna()
        missing_perc = 100*(1-len(temp_data)/len(df[col]))
        ax = plt.subplot(nrows,ncols,i)
        counts_train = temp_data.value_counts().sort_values()
        if len(counts_train)>maxvals:
            counts_train = counts_train[-maxvals:]
        y_postrain = np.array(list(counts_train.index))
        ax.barh(np.arange(1,len(counts_train)+1), counts_train, color='blue', align='center', height=0.4)
        ax.set_yticks(np.arange(1,len(counts_train)+1))
        ax.set_yticklabels(y_postrain)
        ax.set_title('%s (%.2f%s missing)'%(col,missing_perc,'%'), fontsize=20)
        ax.set_ylabel('')
        aic.jupyter.ax_properties(ax)
        i += 1
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=3.0)
    
def plot_histograms_for_columns(df,cols,rowheight=5):
    nrows, ncols = len(cols), 2
    nplots = len(cols)
    fig, ax = plt.subplots(figsize=(20,rowheight*nrows))
    i = 1
    for col in cols:
        temp_data = df[col].dropna()
        missing_perc = 100*(1-len(temp_data)/len(df[col]))
        ax = plt.subplot(nrows,ncols,i)
        ax.hist(temp_data, bins=50)
        ax.set_title('%s (%.2f%s missing)'%(col,missing_perc,'%'), fontsize=20)
        ax.set_ylabel('')
        aic.jupyter.ax_properties(ax)
        i += 1
        ax = plt.subplot(nrows,ncols,i)
        ax.hist(np.log(temp_data[temp_data>0]), bins=50)
        missing_perc = 100*(1-sum(temp_data>0)/len(df[col]))
        ax.set_title('log of %s (%.2f%s missing or <=0)'%(col,missing_perc,'%'), fontsize=20)
        ax.set_ylabel('')
        aic.jupyter.ax_properties(ax)
        i +=1
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=3.0)

## Import data

## Data understanding - EDA

### Descriptive statistics

### Data visualizations